In [1]:
from utils import *
import tensorflow as tf
from keras import backend as K

2022-10-22 19:40:49.812557: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-22 19:40:50.105109: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-22 19:40:50.837841: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/moises/workspace/devel/lib
2022-10-22 19:40:50.837905: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.

Check if keras is using GPU

In [2]:
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))
K._get_available_gpus()

2022-10-22 19:40:51.503312: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-22 19:40:51.563720: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-22 19:40:51.655722: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-22 19:40:51.655911: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5



2022-10-22 19:40:52.330912: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-22 19:40:52.331350: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-22 19:40:52.331462: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-22 19:40:52.331549: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4176 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5


['/device:GPU:0']

2022-10-22 19:40:52.333938: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-22 19:40:52.334079: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-22 19:40:52.334178: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-22 19:40:52.334422: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-22 19:40:52.334525: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from S

Get tickers

In [3]:
tickers_sets_path = 'data/tickers/sets/'

sets_files = sorted([tickers_sets_path + x for x in listdir(tickers_sets_path) if 'json' in x], reverse=False)

tickers_sets = []

for set_file in sets_files:
    with open(set_file, 'r') as f:
        tickers_sets.append(json.load(f))

Read dataset

In [4]:
df = pd.read_csv('data/dataset.csv')
df = df.set_index('Date')
# df = df[:-60]

Define training sets and data size

In [5]:
in_sets = [0,1,2]
out_sets = [2]
data_size = 1000

input_tickers = set()
output_tickers = set()
for in_set in in_sets:
    input_tickers = input_tickers.union(set(tickers_sets[in_set]))

for out_set in out_sets:
    output_tickers = output_tickers.union(set(tickers_sets[out_set]))

input_tickers = list(input_tickers)
output_tickers  = list(output_tickers)


Process data

In [6]:
scaler, x, y, dic = dataa(df, data_size, input_tickers, output_tickers, \
                        step_size=1, input_size=40, output_size=20, \
                        feature_range=(0,1), return_log=False)

493 14


Split dataset i training, validation and test

In [7]:
x_train, y_train, x_val, y_val, x_test, y_test = train_val_test_split(x,y)

Define hyperparameters

In [8]:
input_shape = (x.shape[1],x.shape[2])
output_shape = (y.shape[1], y.shape[2])
layers_info_list = [
    [
        (75, 0.15, True), 
        (75, 0.15, True),
        (75, 0.15, True),
        (75, 0.15, False)
    ],
    [
        (250, 0.20, True), 
        (250, 0.20, True),
        (250, 0.20, True),
        (250, 0.20, False)
    ],
    [
        (600, 0.15, True),
        (600, 0.15, False)
    ],
    [
        (80, 0.10, True), 
        (80, 0.10, True),
        (80, 0.10, False)
    ],
    [
        (450, 0.15, True), 
        (450, 0.15, True),
        (450, 0.15, False)
    ]
]



Create Models

In [9]:
models = []
hyperparameters = range(len(layers_info_list))

for i in hyperparameters:
    layers_info = layers_info_list[i]
    models.append(create_model(input_shape, output_shape, layers_info, metrics=['accuracy', 'mse']))

Train networks

In [10]:
history_list = []
epochs_list = [125, 125, 125, 125, 125]

set_name = 'set2/'
plot_path_dir = 'metrics/' + set_name
extra_info = ''

j=0
for i in hyperparameters:

    model = models[j]
    epochs = epochs_list[j]
    j += 1
    try:
        
        history = model.fit(
            x=x_train,
            y=y_train,
            epochs=epochs,
            batch_size=32,
            validation_data=(x_val, y_val)
        )

        history_list.append(history)

        model_name = 'LSTM' + str(i) + extra_info
        # model.save('models/' + set_name + model_name + '/model')

        plot_path_loss = plot_path_dir + 'LOSS_' + model_name + '.png'
        validation_plot(history.history['loss'], history.history['val_loss'], 'train vs validation loss', f=lambda: plt.savefig(plot_path_loss))

        plot_path_mse = plot_path_dir + 'MSE_' + model_name + '.png'
        validation_plot(history.history['mse'], history.history['val_mse'], 'train vs validation mse', f=lambda: plt.savefig(plot_path_mse), plot_type='mse')

        plot_path_accuracy = plot_path_dir + 'ACCURACY_' + model_name + '.png'
        validation_plot(history.history['accuracy'], history.history['val_accuracy'], 'train vs validation accuracy', f=lambda: plt.savefig(plot_path_accuracy), plot_type='accuracy')

    except:

        print('ERRO NO MODELO ' + str(i))
    

Epoch 1/125


2022-10-22 19:41:00.873305: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8600


21/21 [==============================] - 6s 56ms/step - loss: 0.3841 - accuracy: 0.0976 - mse: 0.1720 - val_loss: 0.2108 - val_accuracy: 0.1745 - val_mse: 0.0497
Epoch 2/125
21/21 [==============================] - 0s 10ms/step - loss: 0.2122 - accuracy: 0.1521 - mse: 0.0480 - val_loss: 0.1734 - val_accuracy: 0.1883 - val_mse: 0.0330
Epoch 3/125
21/21 [==============================] - 0s 10ms/step - loss: 0.1813 - accuracy: 0.1933 - mse: 0.0347 - val_loss: 0.1482 - val_accuracy: 0.2213 - val_mse: 0.0240
Epoch 4/125
21/21 [==============================] - 0s 9ms/step - loss: 0.1608 - accuracy: 0.1940 - mse: 0.0272 - val_loss: 0.1331 - val_accuracy: 0.2457 - val_mse: 0.0190
Epoch 5/125
21/21 [==============================] - 0s 10ms/step - loss: 0.1480 - accuracy: 0.2315 - mse: 0.0229 - val_loss: 0.1237 - val_accuracy: 0.2638 - val_mse: 0.0168
Epoch 6/125
21/21 [==============================] - 0s 10ms/step - loss: 0.1334 - accuracy: 0.2735 - mse: 0.0187 - val_loss: 0.1086 - val_accu

In [11]:
# for model in models:
#     y_predict = model.predict(x_val)
#     mse_ = np.square(np.subtract(y_val,y_predict)).mean(axis=1)
#     var_ = np.var(np.subtract(y_val,y_predict), axis=1)
#     e_ = np.sum(np.mean(np.sqrt(np.add(mse_,var_)),axis=0))
    

In [16]:
scaler2, x2, y2, dic2 = dataa(df, data_size, input_tickers, output_tickers, \
                        step_size=1, input_size=40, output_size=20, \
                        feature_range=(0,1), return_log=True)

493 14


In [17]:
x2_train, y2_train, x2_val, y2_val, x2_test, y2_test = train_val_test_split(x,y)

In [19]:
models2 = []
hyperparameters = range(len(layers_info_list))

for i in hyperparameters:
    layers_info = layers_info_list[i]
    models2.append(create_model(input_shape, output_shape, layers_info, metrics=['accuracy', 'mse']))

In [20]:
history2_list = []
epochs_list = [125, 125, 125, 125, 125]

set_name = 'set2/'
plot_path_dir = 'metrics/' + set_name
extra_info = '_RETURN_LOG'

j=0
for i in hyperparameters:

    model = models2[j]
    epochs = epochs_list[j]
    j += 1
    try:
        
        history = model.fit(
            x=x2_train,
            y=y2_train,
            epochs=epochs,
            batch_size=32,
            validation_data=(x2_val, y2_val)
        )

        history2_list.append(history)

        model_name = 'LSTM' + str(i) + extra_info
        # model.save('models/' + set_name + model_name + '/model')

        plot_path_loss = plot_path_dir + 'LOSS_' + model_name + '.png'
        validation_plot(history.history['loss'], history.history['val_loss'], 'train vs validation loss', f=lambda: plt.savefig(plot_path_loss))

        plot_path_mse = plot_path_dir + 'MSE_' + model_name + '.png'
        validation_plot(history.history['mse'], history.history['val_mse'], 'train vs validation mse', f=lambda: plt.savefig(plot_path_mse), plot_type='mse')

        plot_path_accuracy = plot_path_dir + 'ACCURACY_' + model_name + '.png'
        validation_plot(history.history['accuracy'], history.history['val_accuracy'], 'train vs validation accuracy', f=lambda: plt.savefig(plot_path_accuracy), plot_type='accuracy')

    except:

        print('ERRO NO MODELO ' + str(i))
    

Epoch 1/125
21/21 [==============================] - 6s 60ms/step - loss: 0.3719 - accuracy: 0.1091 - mse: 0.1628 - val_loss: 0.1990 - val_accuracy: 0.1656 - val_mse: 0.0458
Epoch 2/125
21/21 [==============================] - 0s 11ms/step - loss: 0.2050 - accuracy: 0.1591 - mse: 0.0448 - val_loss: 0.1545 - val_accuracy: 0.2248 - val_mse: 0.0258
Epoch 3/125
21/21 [==============================] - 0s 10ms/step - loss: 0.1706 - accuracy: 0.1880 - mse: 0.0307 - val_loss: 0.1332 - val_accuracy: 0.2206 - val_mse: 0.0190
Epoch 4/125
21/21 [==============================] - 0s 10ms/step - loss: 0.1519 - accuracy: 0.2179 - mse: 0.0242 - val_loss: 0.1210 - val_accuracy: 0.2915 - val_mse: 0.0158
Epoch 5/125
21/21 [==============================] - 0s 10ms/step - loss: 0.1400 - accuracy: 0.2719 - mse: 0.0205 - val_loss: 0.1140 - val_accuracy: 0.3046 - val_mse: 0.0140
Epoch 6/125
21/21 [==============================] - 0s 10ms/step - loss: 0.1289 - accuracy: 0.3077 - mse: 0.0175 - val_loss: 0.10